# Evaluation

In [3]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

In [26]:
class MajorLabelClassifier(BaseEstimator, ClassifierMixin):
    
    def fit(self, X, y=None):
        y = list(y)
        self.possible_y_ = set(y)
        self.total_example_ = len(y)
        self.major_label_count_ = 0
        for p in self.possible_y_:
            count = y.count(p)
            if count > self.major_label_count_:
                self.major_label_ = p
                self.major_label_count_ = count
        self.fitted_ = True
        return self
    
    def predict(self, X):
        return np.array([self.major_label_ for i in range(len(X))])


# Load Training Data

In [19]:
train_raw = pd.read_csv('train_data.csv')
train_raw.head()

,Unnamed: 0,kata,sense,kalimat,pos_tags,clean,targetpos_clean,targetpos_ori,targetpos_pos_tag
0,0,cerah,4801,cuaca cerah adalah lazim panjang tahun,NN NN VB NN NN NN Z,cuaca cerah lazim,1,1,1
1,1,cerah,4801,gambar yang hasil oleh layarnya cukup cerah da...,NNP SC VB IN NN RB JJ CC VB NN SC JJ VB NN SC ...,gambar hasil layarnya cerah milik speaker hasi...,3,6,6
2,2,cerah,4803,masa depan yang cerah bagi pemuda umur somenum...,NN NN SC VB IN NN NN CD IN NNP NNP CD Z,cerah bagi pemuda umur prancis abad,0,3,3
3,3,cerah,4801,cor caroli alpha canum venaticorum nama lengka...,NNP NNP Z NNP NNP NNP Z Z Z NN RB VB NNP NNP N...,cor caroli alpha canum venaticorum nama lengka...,12,16,21
4,4,cerah,4801,sanders lebih suka cat air untuk lilo dengan m...,NN RB VB NN NN SC NNP IN NN VB NN NN NN NN NN Z,sanders suka cat air lilo maksud tampil warna ...,8,11,11


In [20]:
ambiguous_word = set(train_raw.kata)

In [22]:
train = dict()
for w in ambiguous_word:
    train[w] = train_raw.query('kata == "{}"'.format(w))
    
train['baru'].head()

,Unnamed: 0,kata,sense,kalimat,pos_tags,clean,targetpos_clean,targetpos_ori,targetpos_pos_tag
1642,1642,baru,3901,johan henri quanjer adalah orang tulis belanda...,NNP NNP NNP VB NND NN NNP Z NN CC NND NN NN JJ...,johan henri quanjer orang tulis belanda terbit...,10,12,13
1643,1643,baru,3901,pada akhir tahun somenumber ken hirai tampil u...,IN NN NN CD Z NNP NNP JJ SC OD RB IN NN NN NN ...,ken hirai tampil kalinya dalam acara malam bar...,7,14,15
1644,1644,baru,3902,bagai contoh dapat cari tahu dengan cepat bera...,IN NN MD VB VB IN JJ WH NN CC FW CD NN IN CD N...,contoh cari cepat gnp gdp negara usaha publik ...,8,19,20
1645,1645,baru,3901,turut kronologi mesir sebekhotep iii adalah ra...,IN NNP NNP Z NNP NNP VB NN NN JJ SC NEG JJ NNP...,kronologi mesir sebekhotep iii raja baru kenal...,5,7,9
1646,1646,baru,3902,namun nama baru lambung pada album dua yang ju...,CC Z RB JJ VB IN NN NN SC VB FW FW FW FW FW Z ...,nama baru lambung album judul a chacun son his...,1,2,3


# Load Test Data

In [6]:
test_raw = pd.read_csv('../testing_data.csv')
test_raw.head()

,id,word,kalimat
0,13,asing,"Para pecinta film indonesia atau tv, pasti tak..."
1,19,asing,Pasti telinga kita merasa asing dan aneh mende...
2,41,asing,Warga negara asing atau warga negara Persemakm...
3,44,asing,"Selama lima belas tahun memerintah, Sultan Mah..."
4,121,asing,Yang kemudian diikuti dengan donat-donat waral...


In [10]:
test = dict()
for w in ambiguous_word:
    test[w] = test_raw.query('word == "{}"'.format(w))
    
test['baru'].head()

,id,word,kalimat
631,76787,baru,"Namun, penelitian terhadap peranan DNA di dala..."
632,77113,baru,Tim Atkinson memiliki pemain baru seperti Jesp...
633,77460,baru,"Dalam usia baru menginjak 15 tahun, Malik tela..."
634,77587,baru,Keduanya bertindak bersama-sama memungkinkan l...
635,77813,baru,Film pertama dari seri ini tayang perdana di E...


# Dummy

In [31]:
dummy_clf = {w: MajorLabelClassifier().fit(train[w], train[w].sense) for w in ambiguous_word}

In [34]:
res_file = open('dummy_baseline_classification.csv', 'w')

for i in range(len(test_raw)):
    row = test_raw.iloc[i]
    res_file.write('{},{},{}'.format(row.id, row.word, dummy_clf[row.word].predict([None])[0]))
    
res_file.close()